In [1]:
import os,sys
sys.path.append('../src')

import MSSP


In [2]:
gdir = '/Users/brandon/Documents/GitHub/SNAP Fisheries Spreadsheets/json/current'
outdir = '/Users/brandon/Documents/GitHub/SNAP Fisheries Spreadsheets/json/2016-06-30'

In [3]:
Eq = MSSP.MsspFromJson(gdir)

### Add N/A answers

In [5]:
help(Eq.refactor_answers)

Help on method refactor_answers in module MSSP.mssp_data_store:

refactor_answers(self, question, answers) method of MSSP.importers.from_json.JsonImporter instance
    Re-arrange or otherwise re-define the answers to a given question.  The input should include a single
    QuestionID, and a new list of the question's answers.  This list must be a SUPERSET of the question's
    existing answers [see merge_answers and delete_answer [TODO] for an antidote] and must be supplied in the
    new correct order.
    
    The method will generate a mapping from the current answers to the new answers, and then will go through
    all the criteria and caveats, re-mapping the index values.
    
    This method is made atomic by performing the mapping onto new data frames that are only installed if
    the operation is successful.
    :param question:
    :param answers:
    :return:



In [14]:
Eq.search('M10')

searchAttributes(attributes=[UUID('2157c264-a1ed-3559-ae32-e1fd5044d055')], questions=[154], targets=[])

In [13]:
Eq.questions(154)[0].valid_answers

['No', 'Yes']

In [15]:
Eq.refactor_answers(154, ['N/A', 'No', 'Yes'])

In [16]:
Eq.search('M33')

searchAttributes(attributes=[UUID('d363b9c1-f764-30c0-b70f-66941e816c5b')], questions=[187], targets=[])

In [18]:
Eq.questions(187)[0].valid_answers

['No', 'Yes']

In [19]:
Eq.refactor_answers(187, ['N/A', 'No', 'Yes'])

### Question 80 - merge answers

In [24]:
Eq.question(80)

Question ID: 80
"If handled or captured and released, is survivorship compromised? (AND change answer to "Yes/No" rather than "high/low")"
(Category: Biophysical/life history)
  * IF LOW
  * IF YES
  * Is species susceptible to barotrauma, and/or has low survivorship on release?
  * NEW
  * Biology/life history
  * If handled or captured and released, is survivorship compromised? 
  * R7
  * reword to "High survicorship on release?"
  * M36
  * ·       Disease/barotrauma:  Reword with less jargon.  If you catch and release, will it survive?
 References: ControlRules Row 15, ControlRules Row 22, Monitoring Col AO
Valid Answers: ['No', 'Yes']
 Satisfied By: None
 Satisfies: None
Has Caveats:
     QuestionID  TargetID AnswerValue  \
412          80        50         Yes   
413          80        51         Yes   
414          80        52         Yes   
415          80        53         Yes   
416          80        54         Yes   
417          80        55         Yes   
418          8

In [22]:
Eq.merge_answers(80,['Yes', 'LOW'], merge_to='Yes')

Merging answers into Yes:
 Yes
 LOW
Really continue? [y]: y


In [23]:
Eq.merge_answers(80,['No', 'HIGH'], merge_to='No')

Merging answers into No:
 No
 HIGH
Really continue? [y]: 


### Question 97 - make composite of 98-106

In [31]:
Eq._questions[97].satisfied_by = set(range(98,107))

In [50]:
Eq._set_satisfies(97)

In [32]:
Eq.question(97)

Question ID: 97
  * Does assessment suggest a certain form of decision rule? (PENDING OUTCOMES OF ASSESSMENT COMPONENT OF MSSP - 2nd layer?)
  * IF YES
 References: ControlRules Row 32
Valid Answers: ['No', 'Yes']
 Satisfied By: [98, 99, 100, 101, 102, 103, 104, 105, 106]
 Satisfies: None
Has Caveats:
     QuestionID  TargetID AnswerValue  \
787          97        50         Yes   
788          97        51         Yes   
789          97        52         Yes   
790          97        53         Yes   
791          97        54         Yes   
792          97        55         Yes   
793          97        58         Yes   
794          97        59         Yes   
795          97        60         Yes   
796          97        61         Yes   
797          97        64         Yes   
798          97        88         Yes   

                                                 Note  
787                            YES - if catch-related  
788                            YES - if catch-relat

### Question 110 - add title

In [45]:
Eq._attributes['88bf52a1-cb69-3820-848f-3a3cdfbca583'].text = 'Is there latent effort in the fishery?'

In [46]:
Eq.attributes(Eq.attributes_for(110))

AttributeID ec11bea9-cbd3-3736-a89d-bbc7403e6910: IF YES
AttributeID 2eb34970-1ca5-340c-b602-fde28fc7a841: Operational
AttributeID 88bf52a1-cb69-3820-848f-3a3cdfbca583: Is there latent effort in the fishery?
AttributeID 7b7dfde6-41b3-3745-80ad-a433ada48fde: R35


### Question 120 - make into two questions
put the new one at QID 130, which is blank

This is tricky- need to:
 1. clone question into Eq._questions
 2. remap attributes by visual inspection
 3. except for category- duplicate
 4. create new title for new question
 5. remap all caveats with Question ID 120 to Question ID 130 (everything splits easily criteria vs caveats)

In [48]:
Eq._questions[120].__dict__

{'_external_answers': [u'low', u'1', u'2', u'moderate', u'3', u'4', u'high'],
 '_yes_no_included': False,
 'category': UUID('ceb4663a-e214-3c99-a3f4-1e49fedb3763'),
 'references': [('ControlRules', (59, None)),
  ('Monitoring', (None, 8)),
  ('ControlRules', (60, None)),
  ('ControlRules', (61, None))],
 'satisfied_by': set(),
 'satisfies': set(),
 'title': UUID('c661691a-999d-3a16-87bc-19506b938659'),
 'valid_answers': [u'low', u'1', u'2', u'moderate', u'3', u'4', u'high']}

In [53]:
q120 = Eq._questions[120]

In [51]:
q130 = MSSP.mssp_objects.MsspQuestion()

In [54]:
import copy

In [55]:
q130._external_answers = copy.copy(q120._external_answers)
q130.category = copy.copy(q120.category)
q130.references = [('Monitoring', (None,8))]
q130._update_valid_answers()

In [59]:
q130.title = Eq._attributes.add_element('How culturally engrained is cooperation to management?')

In [61]:
Eq._questions[130] = q130

In [65]:
Eq.add_attribute_mapping(130, q130.category)

attribute map - 1 found


MsspError: QID 130: Attribute mapping already exists (General)

In [70]:
import uuid

In [73]:
for x in ['4ea136c3-94aa-3dda-85b9-ab3c3d07f8ca', '3fc6b453-876e-3792-9187-a562235b3754', 'fb85634b-9d29-3369-b134-0b0215ba9e7d', 'b08347fc-d4fd-38d6-b9b4-a4a6c36c1cac', '5363bef5-825a-34c1-9e1c-f70551a5c2d9']:
    Eq.add_attribute_mapping(130, uuid.UUID(x))
    Eq.del_attribute_mapping(120, uuid.UUID(x))

Removed 1 reference
attribute map - 1 found


MsspError: QID 130: Attribute mapping already exists (criteria)

In [75]:
Eq.attributes(Eq.attributes_for(130))

AttributeID c48b9914-3752-3d85-94ac-917636e10ee5: How culturally engrained is cooperation to management?
AttributeID ceb4663a-e214-3c99-a3f4-1e49fedb3763: General
AttributeID 3fc6b453-876e-3792-9187-a562235b3754: criteria
AttributeID fb85634b-9d29-3369-b134-0b0215ba9e7d: Fishery cooperation (wrt HS approach)
AttributeID b08347fc-d4fd-38d6-b9b4-a4a6c36c1cac: How culturally ingrained is cooperation to management? Categories 1) unwilling to share and unwilling to record, 2) willing to share info but unwilling to record info, 3) willing to share and record but may not do so reliably, 4) willing and reliable sharing and recording
AttributeID 5363bef5-825a-34c1-9e1c-f70551a5c2d9: CRITERIA: begin here. These will provide a first-cut shortlist of monitoring options based on the MINIMUM requirements for each form of monitoring.
AttributeID 4ea136c3-94aa-3dda-85b9-ab3c3d07f8ca: M3


In [79]:
Eq._criteria.loc[Eq._criteria['QuestionID']==120, 'QuestionID'] = 130

In [83]:
Eq.delete_answer(120, '1')
Eq.delete_answer(120, '2')
Eq.delete_answer(120, '3')
Eq.delete_answer(120, '4')
Eq.delete_answer(130, 'low')
Eq.delete_answer(130, 'moderate')
Eq.delete_answer(130, 'high')

In [85]:
Eq.question(120)

Question ID: 120
"What is the level of fishery cooperation, in terms of complying with and supporting management measures?"
(Category: General)
  * Level of fisher cooperation (includes cultural preferences)
  * low
  * Socio-economic
  * moderate
  * high
  * R44
  * includes cultural preferences
  * How culturally ingrained is cooperation to management?
  * willing if paid
  * Are there enabling conditions
 References: ControlRules Row 59, ControlRules Row 60, ControlRules Row 61, Monitoring Col H
Valid Answers: [u'low', u'moderate', u'high']
 Satisfied By: None
 Satisfies: None
Has Caveats:
      QuestionID  TargetID AnswerValue                                   Note
1089         120        50         low                                     NO
1090         120        50    moderate                                CAUTION
1091         120        50        high                                    YES
1092         120        51         low                                     NO
1093     

### Question 125: merge answers

In [87]:
Eq.merge_answers(125, ['Yes', 'open access'], merge_to='open access')

Merging answers into open access:
 Yes
 open access
Really continue? [y]: 


In [90]:
Eq.question(125)

Question ID: 125
"Is the fishery open access, as opposed to limited entry?"
(Category: General)
  * intermediate
  * Governance
  * Open access governance, to limited entry. (e.g.in an open access governance system the efficacy of tools like size , fishing season, or effort limits is often reduced by effort creep, but when access is limited this seems to be less of a problem and so the same tool would look more attractive)
  * open access
  * limited entry
  * Is the fishery open access? (as opposed to limited entry)
  * IF YES
  * CAVEATS - Second, refine the shortlist of monitoring options according to the below range of criteria
  * Strength of governance
  * R48
  * Check consistency in monitoring spreadsheet, question M8. In open access governance system, the efficacy of tools like size, fishng season, or effort limits is often reduced by effort creep, but when access is limited this seems to be less of a problem nd so the same tool would look more attractive.
  * M8
  * as oppose

In [89]:
Eq.delete_answer(125, 'No')

Matching Caveats:
      Answer                                NoteID  QuestionID  TargetID
1308       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        50
1311       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        51
1314       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        52
1317       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        53
1320       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        54
1323       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        55
1326       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        56
1329       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        57
1332       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        58
1335       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        59
1338       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        60
1341       0  a79fbfc9-9ac6-330d-bd95-401ba5d4c868         125        61
1344       0  7ef90e24-35a1-3646-